In [1]:
import duckdb

In [2]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [3]:
con.execute("PRAGMA show_tables;").fetchall()

[('bronze_produtos',), ('produtos',)]

In [4]:
df = con.execute("""
SELECT *
FROM( 
    SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row
    FROM bronze_produtos
    WHERE data_ingestao >= '2025-07-17'
) WHERE row = 1         
""").fetchdf()
df.head(10)

,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao,row
0,1001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-07-17 09:08:33.382544,1
1,1003,PREGO,BT10,100,60,z0019_2.csv,2025-07-17 09:28:13.014940,1
2,1004,SERRA,BT50,100,200,z0019_2.csv,2025-07-17 09:28:13.014940,1
3,1002,MARTELO,BT50,100,1500,z0019_1.csv,2025-07-17 09:08:33.382544,1
4,1005,MACHADO,BT50,100,100,z0019_2.csv,2025-07-17 09:28:13.014940,1


In [5]:
df.head(10)

,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao,row
0,1001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-07-17 09:08:33.382544,1
1,1003,PREGO,BT10,100,60,z0019_2.csv,2025-07-17 09:28:13.014940,1
2,1004,SERRA,BT50,100,200,z0019_2.csv,2025-07-17 09:28:13.014940,1
3,1002,MARTELO,BT50,100,1500,z0019_1.csv,2025-07-17 09:08:33.382544,1
4,1005,MACHADO,BT50,100,100,z0019_2.csv,2025-07-17 09:28:13.014940,1


In [6]:
df_final = df.drop(columns=['nome_arquivo', 'data_ingestao', 'row'])
df_final = df_final.rename(columns={"NATBR": "id"})
df_final = df_final.rename(columns={"MAKTX": "nm_produto"})
df_final = df_final.rename(columns={"WERKS": "id_categoria"})
df_final = df_final.rename(columns={"MAINS": "id_fornecedor"})
df_final = df_final.rename(columns={"LABST": "id_preco"})
df_final.head(10)

,id,nm_produto,id_categoria,id_fornecedor,id_preco
0,1001,PARAFUSO,BT10,100,100
1,1003,PREGO,BT10,100,60
2,1004,SERRA,BT50,100,200
3,1002,MARTELO,BT50,100,1500
4,1005,MACHADO,BT50,100,100


In [7]:
df2 = df_final.copy()


In [8]:
df2 = df2.astype(
    {
        "id": int,
        "nm_produto": str,
        "id_categoria": str,
        "id_fornecedor": int,
        "id_preco": float
    }
)
df2.dtypes

id                 int32
nm_produto        object
id_categoria      object
id_fornecedor      int32
id_preco         float64
dtype: object

In [9]:
con.execute("""
CREATE TABLE IF NOT EXISTS produtos(
            id BIGINT,
            nm_produto TEXT,
            id_categoria TEXT,
            id_fornecedor BIGINT,
            vl_preco FLOAT
            )
            
""")

In [39]:
df2.head()

,id,nm_produto,id_categoria,id_fornecedor,id_preco
0,1002,MARTELO,BT50,100,1500.0
1,1005,MACHADO,BT50,100,100.0
2,1001,PARAFUSO,BT10,100,100.0
3,1003,PREGO,BT10,100,60.0
4,1004,SERRA,BT50,100,200.0


In [10]:
con.execute("INSERT INTO produtos SELECT * FROM df2").fetchdf()

,Count
0,5


In [11]:
df_resultado = con.execute("""
SELECT *
FROM produtos
""").fetchdf()
df_resultado.head()

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,1004,SERRA,BT50,100,200.0
1,1005,MACHADO,BT50,100,100.0
2,1002,MARTELO,BT50,100,1500.0
3,1001,PARAFUSO,BT10,100,100.0
4,1003,PREGO,BT10,100,60.0


In [1]:
con.close()

NameError: name 'con' is not defined